In [7]:
import pandas as pd
import numpy as np
import time
import collections
import plotly.graph_objects as go
import plotly.express as px

In [8]:
# Read the printify products excel file into pandas dataframe
orders = pd.read_excel('Printify_Data_Analyst_homework.xlsx', sheet_name = 'Orders')
orders.head(10)

,MERCHANT_ID,ORDER_ID,SHOP_ID,ADDRESS_TO_COUNTRY,ADDRESS_TO_REGION,ORDER_DT,FULFILLED_DT,REPRINT_FLAG,SALES_CHANNEL_TYPE_ID,TOTAL_COST,TOTAL_SHIPPING,MERCHANT_REGISTERED_DT,SUB_IS_ACTIVE_FLAG,SUB_PLAN,SHIPMENT_CARRIER,SHIPMENT_DELIVERD_DT
0,7143578,2.013244e+06,2013244,US,WA,2020-10-02 19:30:15,2020-10-20 07:53:06,NaN,9,2902,2200,2020-05-19 15:22:57,True,business_account_3,UPS,2020-10-08 21:07:29
1,5888534,1.358430e+06,1358429,US,MN,2020-11-21 05:19:03,2020-11-23 00:23:32,NaN,4,1149,924,2018-02-08 07:34:02,True,business_account_3,USPS,2020-11-28 21:50:56
2,6484224,9.393902e+05,939390,US,AK,2020-10-25 01:39:08,2020-10-26 20:30:53,NaN,1,977,400,2019-04-09 20:02:46,True,business_account_3,USPS,2020-11-02 23:37:54
3,7251033,1.856107e+06,1856107,US,PA,2020-10-08 16:35:21,2020-10-10 15:02:47,NaN,1,661,800,2020-07-20 17:04:41,True,business_account_3,UPS,2020-10-15 18:08:37
4,7241488,1.843904e+06,1843904,US,CO,2020-11-17 19:30:10,2020-11-18 03:45:25,NaN,2,719,424,2020-07-15 18:47:51,True,business_account_3,USPS,2020-11-24 00:44:56
5,6949441,1.477206e+06,1477206,US,NV,2020-10-07 05:09:46,2020-10-08 20:33:40,NaN,1,805,400,2019-12-27 22:55:01,True,Plan 4,USPS,2020-10-12 21:44:56
6,7146390,1.720307e+06,1720307,US,CO,2020-11-13 22:00:23,2020-11-15 15:52:22,NaN,1,3010,1024,2020-05-21 14:45:00,True,business_account_3,USPS,2020-11-19 20:41:30
7,6179493,6.044341e+05,604434,US,GA,2020-10-31 02:38:32,2020-11-04 20:51:54,NaN,4,1306,700,2018-10-28 05:11:39,True,Plan 4,USPS,NaT
8,7110135,1.813012e+06,1813012,US,IN,2020-11-13 04:55:57,2020-11-14 01:20:14,NaN,4,1602,624,2020-04-27 00:42:50,True,business_account_3,USPS,2020-11-19 22:49:58
9,5900409,1.325831e+06,1325831,BE,NaN,2020-11-11 11:19:05,2020-11-12 02:44:37,NaN,1,2880,1180,2018-02-23 06:30:55,True,Plan 4,NaN,NaT


In [9]:
# Read the printify products excel file into pandas dataframe
line_items = pd.read_excel('Printify_Data_Analyst_homework.xlsx', sheet_name = 'Line items')
line_items.head(10)

,ORDER_ID,PRINT_PROVIDER_ID,PRODUCT_BRAND,PRODUCT_TYPE,ITEM_STATUS,QUANTITY
0,1.750015e+06,30,Generic brand,Mug,on-hold,3
1,1.750015e+06,30,Generic brand,Mug,on-hold,2
2,1.225155e+06,16,Gildan,Kids clothes,shipment_delivered,1
3,1.828891e+06,39,Bella+Canvas,T-Shirt,shipment_delivered,1
4,1.847848e+06,10,Generic brand,Accessories,shipment_delivered,1
5,1.750016e+06,30,Generic brand,Mug,on-hold,1
6,4.668482e+05,10,Generic brand,Bags,shipment_delivered,1
7,1.569960e+06,45,Gildan,Sweatshirt,fulfilled,1
8,2.004430e+06,29,Gildan,Sweatshirt,shipment_delivered,1
9,2.255664e+05,25,Bella+Canvas,T-Shirt,shipment_delivered,1


In [10]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13506 entries, 0 to 13505
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   MERCHANT_ID             13506 non-null  int64         
 1   ORDER_ID                13506 non-null  float64       
 2   SHOP_ID                 13506 non-null  int64         
 3   ADDRESS_TO_COUNTRY      13505 non-null  object        
 4   ADDRESS_TO_REGION       12930 non-null  object        
 5   ORDER_DT                13506 non-null  datetime64[ns]
 6   FULFILLED_DT            12637 non-null  datetime64[ns]
 7   REPRINT_FLAG            103 non-null    float64       
 8   SALES_CHANNEL_TYPE_ID   13506 non-null  int64         
 9   TOTAL_COST              13506 non-null  int64         
 10  TOTAL_SHIPPING          13506 non-null  int64         
 11  MERCHANT_REGISTERED_DT  13506 non-null  datetime64[ns]
 12  SUB_IS_ACTIVE_FLAG      13506 non-null  bool  

### Oders info() inferences
The merchant_id colum is the main attribute of the entire table. For all the unique merchant_id in the column, that is a valid data entry with or without other attributes. Infact, the lack of some attribute for a merchant_id is an attribute itself e.g. if an merchant_id entry lacks SHIPMENT_DELIVERD_DT means the order have not been delivered yet. 

Nonetheless, Order_id should all be unique since since relationship of order-to-merchant can not be many-to-one although the vice versa is true.

In [11]:
# Drop any row[index] where 'all' values are none
orders.dropna(axis='index', how='any', subset=['MERCHANT_ID', 'ORDER_ID',
                                               'SHOP_ID','ORDER_DT', 
                                              'SALES_CHANNEL_TYPE_ID',
                                              'ADDRESS_TO_COUNTRY',
                                              'TOTAL_COST'])

,MERCHANT_ID,ORDER_ID,SHOP_ID,ADDRESS_TO_COUNTRY,ADDRESS_TO_REGION,ORDER_DT,FULFILLED_DT,REPRINT_FLAG,SALES_CHANNEL_TYPE_ID,TOTAL_COST,TOTAL_SHIPPING,MERCHANT_REGISTERED_DT,SUB_IS_ACTIVE_FLAG,SUB_PLAN,SHIPMENT_CARRIER,SHIPMENT_DELIVERD_DT
0,7143578,2.013244e+06,2013244,US,WA,2020-10-02 19:30:15,2020-10-20 07:53:06,NaN,9,2902,2200,2020-05-19 15:22:57,True,business_account_3,UPS,2020-10-08 21:07:29
1,5888534,1.358430e+06,1358429,US,MN,2020-11-21 05:19:03,2020-11-23 00:23:32,NaN,4,1149,924,2018-02-08 07:34:02,True,business_account_3,USPS,2020-11-28 21:50:56
2,6484224,9.393902e+05,939390,US,AK,2020-10-25 01:39:08,2020-10-26 20:30:53,NaN,1,977,400,2019-04-09 20:02:46,True,business_account_3,USPS,2020-11-02 23:37:54
3,7251033,1.856107e+06,1856107,US,PA,2020-10-08 16:35:21,2020-10-10 15:02:47,NaN,1,661,800,2020-07-20 17:04:41,True,business_account_3,UPS,2020-10-15 18:08:37
4,7241488,1.843904e+06,1843904,US,CO,2020-11-17 19:30:10,2020-11-18 03:45:25,NaN,2,719,424,2020-07-15 18:47:51,True,business_account_3,USPS,2020-11-24 00:44:56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13501,7562898,2.225081e+06,2225081,CA,ON,2020-11-05 21:12:29,NaT,NaN,1,0,0,2020-10-29 16:51:47,False,NaN,NaN,NaT
13502,7475289,2.153683e+06,2153683,US,CO,2020-11-16 04:11:43,NaT,NaN,1,1447,474,2020-09-30 23:11:41,True,business_account_3,NaN,NaT
13503,7144356,1.717608e+06,1717607,US,CT,2020-12-25 00:08:07,NaT,NaN,1,2187,824,2020-05-20 01:38:41,False,NaN,NaN,NaT
13504,7581194,2.243421e+06,2243420,US,IN,2020-11-19 16:43:04,NaT,NaN,4,2187,824,2020-11-05 20:02:09,False,NaN,NaN,NaT


In [12]:
len(orders[orders['ADDRESS_TO_COUNTRY'] == 'US'])

11816

In [13]:
# Unique merchants on printify
print(len(orders['MERCHANT_ID'].unique()))

4660


In [14]:
# Total Number of orders
print(len(orders['ORDER_ID'].unique()))

"""
Order ID cannot repeat since an order can only come from one merchant.
13506 - 13495 = 11 orders are repeating
"""

13495


'\nOrder ID cannot repeat since an order can only come from one merchant.\n13506 - 13495 = 11 orders are repeating\n'

In [15]:
# First rows encounter with duplicated order_id 
duplicate_orders1 = orders[orders['ORDER_ID'].duplicated(keep='last')]
duplicate_orders1

,MERCHANT_ID,ORDER_ID,SHOP_ID,ADDRESS_TO_COUNTRY,ADDRESS_TO_REGION,ORDER_DT,FULFILLED_DT,REPRINT_FLAG,SALES_CHANNEL_TYPE_ID,TOTAL_COST,TOTAL_SHIPPING,MERCHANT_REGISTERED_DT,SUB_IS_ACTIVE_FLAG,SUB_PLAN,SHIPMENT_CARRIER,SHIPMENT_DELIVERD_DT
13,6314037,7.521543e+05,752154,US,WA,2020-10-03 13:05:53,2020-10-07 20:20:59,NaN,2,14732,3884,2019-01-25 17:38:56,False,business_account_3,USPS,2020-10-09 21:38:49
537,5868230,3.635962e+05,363596,US,VI,2020-11-18 19:36:02,2020-11-19 15:23:46,NaN,2,9533,2616,2018-01-16 11:29:38,True,business_account_3,USPS,2020-11-24 17:34:42
1534,7439351,2.078441e+06,2078441,US,IA,2020-10-03 21:37:34,2020-10-20 08:02:12,NaN,2,3664,1700,2020-09-23 23:42:43,False,NaN,USPS,2020-10-08 19:47:38
4722,7559003,2.229889e+06,2229889,FR,NaN,2020-12-03 02:27:39,2020-12-09 17:15:58,NaN,6,333,1000,2020-10-28 09:58:01,True,business_account_3,NaN,NaT
5285,7016547,1.555863e+06,1555863,US,LA,2020-10-11 01:33:06,2020-10-14 21:42:00,NaN,1,103,350,2020-02-17 23:36:26,True,business_account_3,NaN,NaT
5436,6118692,5.160665e+05,516066,US,MD,2020-10-11 04:30:55,2020-10-13 01:11:15,NaN,1,1979,850,2018-09-13 12:23:50,True,business_account_3,USPS,2020-10-16 19:44:40
6069,7223323,1.820693e+06,1820693,US,SC,2020-11-04 04:20:21,2020-11-09 18:34:04,NaN,3,6855,2624,2020-07-06 19:27:58,True,business_account_3,USPS,2020-11-09 22:02:28
7321,7476675,2.119858e+06,2119857,US,OH,2020-11-25 07:39:40,2020-11-26 20:37:35,NaN,4,609,424,2020-10-01 08:44:55,False,NaN,USPS,2020-12-08 16:33:58
8099,7368224,1.996849e+06,1996849,US,NC,2020-12-27 12:42:36,2020-12-29 22:49:17,NaN,4,491,424,2020-09-03 16:42:53,True,business_account_3,USPS,2021-01-08 18:39:25
10478,7572140,2.232641e+06,2232640,US,IN,2020-12-07 18:36:13,NaT,NaN,2,0,0,2020-11-02 15:41:56,True,business_account_3,NaN,NaT


In [16]:
# Second rows encounter with duplicated order_id 
duplicate_orders2 = orders[orders['ORDER_ID'].duplicated(keep='first')]
duplicate_orders2

,MERCHANT_ID,ORDER_ID,SHOP_ID,ADDRESS_TO_COUNTRY,ADDRESS_TO_REGION,ORDER_DT,FULFILLED_DT,REPRINT_FLAG,SALES_CHANNEL_TYPE_ID,TOTAL_COST,TOTAL_SHIPPING,MERCHANT_REGISTERED_DT,SUB_IS_ACTIVE_FLAG,SUB_PLAN,SHIPMENT_CARRIER,SHIPMENT_DELIVERD_DT
14,6314037,7.521543e+05,752154,US,WA,2020-10-03 13:05:53,2020-10-07 20:20:59,NaN,2,14732,3884,2019-01-25 17:38:56,False,business_account_3,NaN,NaT
538,5868230,3.635962e+05,363596,US,VI,2020-11-18 19:36:02,2020-11-19 15:23:46,NaN,2,9533,2616,2018-01-16 11:29:38,True,business_account_3,USPS,2020-11-28 16:51:06
1535,7439351,2.078441e+06,2078441,US,IA,2020-10-03 21:37:34,2020-10-20 08:02:12,NaN,2,3664,1700,2020-09-23 23:42:43,False,NaN,USPS,2020-10-20 08:11:09
5437,6118692,5.160665e+05,516066,US,MD,2020-10-11 04:30:55,2020-10-13 01:11:15,NaN,1,1979,850,2018-09-13 12:23:50,True,business_account_3,UPS_MAIL_INNOVATIONS,2020-10-19 19:21:56
5851,7559003,2.229889e+06,2229889,FR,NaN,2020-11-12 01:43:43,2020-11-21 08:40:59,NaN,6,478,800,2020-10-28 09:58:01,True,business_account_3,unknown,NaT
6070,7223323,1.820693e+06,1820693,US,SC,2020-11-04 04:20:21,2020-11-09 18:34:04,NaN,3,6855,2624,2020-07-06 19:27:58,True,business_account_3,USPS,2020-11-13 20:28:54
7502,7016547,1.555863e+06,1555863,US,TX,2020-10-29 22:10:53,2020-11-02 18:07:59,NaN,1,167,350,2020-02-17 23:36:26,True,business_account_3,USPS,2020-11-05 20:05:27
9320,7368224,1.996849e+06,1996849,US,MD,2020-11-13 21:20:55,2020-11-15 15:34:18,NaN,4,1070,524,2020-09-03 16:42:53,True,business_account_3,USPS,2020-11-18 21:06:22
10479,7572140,2.232641e+06,2232640,US,IN,2020-11-05 18:13:25,NaT,NaN,2,0,0,2020-11-02 15:41:56,True,business_account_3,NaN,NaT
10990,7383862,2.015444e+06,2015444,US,PA,2020-12-24 13:53:24,2021-01-21 23:56:17,NaN,2,28172,7480,2020-09-09 21:25:29,True,business_account_3,USPS,NaT


In [17]:
# Combine the duplicated rows into one dataframe for closer comparison
duplicates = pd.concat([duplicate_orders1, duplicate_orders2]).sort_values(by=['MERCHANT_ID'])
duplicates

,MERCHANT_ID,ORDER_ID,SHOP_ID,ADDRESS_TO_COUNTRY,ADDRESS_TO_REGION,ORDER_DT,FULFILLED_DT,REPRINT_FLAG,SALES_CHANNEL_TYPE_ID,TOTAL_COST,TOTAL_SHIPPING,MERCHANT_REGISTERED_DT,SUB_IS_ACTIVE_FLAG,SUB_PLAN,SHIPMENT_CARRIER,SHIPMENT_DELIVERD_DT
537,5868230,3.635962e+05,363596,US,VI,2020-11-18 19:36:02,2020-11-19 15:23:46,NaN,2,9533,2616,2018-01-16 11:29:38,True,business_account_3,USPS,2020-11-24 17:34:42
538,5868230,3.635962e+05,363596,US,VI,2020-11-18 19:36:02,2020-11-19 15:23:46,NaN,2,9533,2616,2018-01-16 11:29:38,True,business_account_3,USPS,2020-11-28 16:51:06
5436,6118692,5.160665e+05,516066,US,MD,2020-10-11 04:30:55,2020-10-13 01:11:15,NaN,1,1979,850,2018-09-13 12:23:50,True,business_account_3,USPS,2020-10-16 19:44:40
5437,6118692,5.160665e+05,516066,US,MD,2020-10-11 04:30:55,2020-10-13 01:11:15,NaN,1,1979,850,2018-09-13 12:23:50,True,business_account_3,UPS_MAIL_INNOVATIONS,2020-10-19 19:21:56
13,6314037,7.521543e+05,752154,US,WA,2020-10-03 13:05:53,2020-10-07 20:20:59,NaN,2,14732,3884,2019-01-25 17:38:56,False,business_account_3,USPS,2020-10-09 21:38:49
14,6314037,7.521543e+05,752154,US,WA,2020-10-03 13:05:53,2020-10-07 20:20:59,NaN,2,14732,3884,2019-01-25 17:38:56,False,business_account_3,NaN,NaT
7502,7016547,1.555863e+06,1555863,US,TX,2020-10-29 22:10:53,2020-11-02 18:07:59,NaN,1,167,350,2020-02-17 23:36:26,True,business_account_3,USPS,2020-11-05 20:05:27
5285,7016547,1.555863e+06,1555863,US,LA,2020-10-11 01:33:06,2020-10-14 21:42:00,NaN,1,103,350,2020-02-17 23:36:26,True,business_account_3,NaN,NaT
6069,7223323,1.820693e+06,1820693,US,SC,2020-11-04 04:20:21,2020-11-09 18:34:04,NaN,3,6855,2624,2020-07-06 19:27:58,True,business_account_3,USPS,2020-11-09 22:02:28
6070,7223323,1.820693e+06,1820693,US,SC,2020-11-04 04:20:21,2020-11-09 18:34:04,NaN,3,6855,2624,2020-07-06 19:27:58,True,business_account_3,USPS,2020-11-13 20:28:54


In [18]:
# Return order
duplicates[:2]

,MERCHANT_ID,ORDER_ID,SHOP_ID,ADDRESS_TO_COUNTRY,ADDRESS_TO_REGION,ORDER_DT,FULFILLED_DT,REPRINT_FLAG,SALES_CHANNEL_TYPE_ID,TOTAL_COST,TOTAL_SHIPPING,MERCHANT_REGISTERED_DT,SUB_IS_ACTIVE_FLAG,SUB_PLAN,SHIPMENT_CARRIER,SHIPMENT_DELIVERD_DT
537,5868230,363596.227,363596,US,VI,2020-11-18 19:36:02,2020-11-19 15:23:46,NaN,2,9533,2616,2018-01-16 11:29:38,True,business_account_3,USPS,2020-11-24 17:34:42
538,5868230,363596.227,363596,US,VI,2020-11-18 19:36:02,2020-11-19 15:23:46,NaN,2,9533,2616,2018-01-16 11:29:38,True,business_account_3,USPS,2020-11-28 16:51:06


In [19]:
# Different carrier na deliverly date. Return order different carrier 
# hypothesis USPS messed the package and another carrier was selected.
duplicates[2:4]

,MERCHANT_ID,ORDER_ID,SHOP_ID,ADDRESS_TO_COUNTRY,ADDRESS_TO_REGION,ORDER_DT,FULFILLED_DT,REPRINT_FLAG,SALES_CHANNEL_TYPE_ID,TOTAL_COST,TOTAL_SHIPPING,MERCHANT_REGISTERED_DT,SUB_IS_ACTIVE_FLAG,SUB_PLAN,SHIPMENT_CARRIER,SHIPMENT_DELIVERD_DT
5436,6118692,516066.48428,516066,US,MD,2020-10-11 04:30:55,2020-10-13 01:11:15,NaN,1,1979,850,2018-09-13 12:23:50,True,business_account_3,USPS,2020-10-16 19:44:40
5437,6118692,516066.48428,516066,US,MD,2020-10-11 04:30:55,2020-10-13 01:11:15,NaN,1,1979,850,2018-09-13 12:23:50,True,business_account_3,UPS_MAIL_INNOVATIONS,2020-10-19 19:21:56


In [20]:
# One is delivered but the other is yet. Remove the delivered
# Return order that is not yet delivered
duplicates[4:6]

,MERCHANT_ID,ORDER_ID,SHOP_ID,ADDRESS_TO_COUNTRY,ADDRESS_TO_REGION,ORDER_DT,FULFILLED_DT,REPRINT_FLAG,SALES_CHANNEL_TYPE_ID,TOTAL_COST,TOTAL_SHIPPING,MERCHANT_REGISTERED_DT,SUB_IS_ACTIVE_FLAG,SUB_PLAN,SHIPMENT_CARRIER,SHIPMENT_DELIVERD_DT
13,6314037,752154.3,752154,US,WA,2020-10-03 13:05:53,2020-10-07 20:20:59,NaN,2,14732,3884,2019-01-25 17:38:56,False,business_account_3,USPS,2020-10-09 21:38:49
14,6314037,752154.3,752154,US,WA,2020-10-03 13:05:53,2020-10-07 20:20:59,NaN,2,14732,3884,2019-01-25 17:38:56,False,business_account_3,NaN,NaT


In [21]:
# Seems different, only need to change ORDER_ID for one
duplicates[6:8]

,MERCHANT_ID,ORDER_ID,SHOP_ID,ADDRESS_TO_COUNTRY,ADDRESS_TO_REGION,ORDER_DT,FULFILLED_DT,REPRINT_FLAG,SALES_CHANNEL_TYPE_ID,TOTAL_COST,TOTAL_SHIPPING,MERCHANT_REGISTERED_DT,SUB_IS_ACTIVE_FLAG,SUB_PLAN,SHIPMENT_CARRIER,SHIPMENT_DELIVERD_DT
7502,7016547,1555863.282,1555863,US,TX,2020-10-29 22:10:53,2020-11-02 18:07:59,NaN,1,167,350,2020-02-17 23:36:26,True,business_account_3,USPS,2020-11-05 20:05:27
5285,7016547,1555863.282,1555863,US,LA,2020-10-11 01:33:06,2020-10-14 21:42:00,NaN,1,103,350,2020-02-17 23:36:26,True,business_account_3,NaN,NaT


In [22]:
# Return order
duplicates[8:10]

,MERCHANT_ID,ORDER_ID,SHOP_ID,ADDRESS_TO_COUNTRY,ADDRESS_TO_REGION,ORDER_DT,FULFILLED_DT,REPRINT_FLAG,SALES_CHANNEL_TYPE_ID,TOTAL_COST,TOTAL_SHIPPING,MERCHANT_REGISTERED_DT,SUB_IS_ACTIVE_FLAG,SUB_PLAN,SHIPMENT_CARRIER,SHIPMENT_DELIVERD_DT
6069,7223323,1820693.162,1820693,US,SC,2020-11-04 04:20:21,2020-11-09 18:34:04,NaN,3,6855,2624,2020-07-06 19:27:58,True,business_account_3,USPS,2020-11-09 22:02:28
6070,7223323,1820693.162,1820693,US,SC,2020-11-04 04:20:21,2020-11-09 18:34:04,NaN,3,6855,2624,2020-07-06 19:27:58,True,business_account_3,USPS,2020-11-13 20:28:54


## Duplicate rows shown above

The unique value of order_id was less than all merchants meaning there were duplicates. 11 of them we caught.
Notice how some follow each other i.e. 13, 14 and 537, 538 etc 

### But what if they are returned orders? 
#### - That is an assumption made but more investigations are needed

In [23]:
# Remove all 2nd duplicated rows and reset the index of the df.
for index in duplicate_orders2.index:
    orders.drop(index, inplace=True)
orders.reset_index(drop=True, inplace=True)
orders

,MERCHANT_ID,ORDER_ID,SHOP_ID,ADDRESS_TO_COUNTRY,ADDRESS_TO_REGION,ORDER_DT,FULFILLED_DT,REPRINT_FLAG,SALES_CHANNEL_TYPE_ID,TOTAL_COST,TOTAL_SHIPPING,MERCHANT_REGISTERED_DT,SUB_IS_ACTIVE_FLAG,SUB_PLAN,SHIPMENT_CARRIER,SHIPMENT_DELIVERD_DT
0,7143578,2.013244e+06,2013244,US,WA,2020-10-02 19:30:15,2020-10-20 07:53:06,NaN,9,2902,2200,2020-05-19 15:22:57,True,business_account_3,UPS,2020-10-08 21:07:29
1,5888534,1.358430e+06,1358429,US,MN,2020-11-21 05:19:03,2020-11-23 00:23:32,NaN,4,1149,924,2018-02-08 07:34:02,True,business_account_3,USPS,2020-11-28 21:50:56
2,6484224,9.393902e+05,939390,US,AK,2020-10-25 01:39:08,2020-10-26 20:30:53,NaN,1,977,400,2019-04-09 20:02:46,True,business_account_3,USPS,2020-11-02 23:37:54
3,7251033,1.856107e+06,1856107,US,PA,2020-10-08 16:35:21,2020-10-10 15:02:47,NaN,1,661,800,2020-07-20 17:04:41,True,business_account_3,UPS,2020-10-15 18:08:37
4,7241488,1.843904e+06,1843904,US,CO,2020-11-17 19:30:10,2020-11-18 03:45:25,NaN,2,719,424,2020-07-15 18:47:51,True,business_account_3,USPS,2020-11-24 00:44:56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13490,7562898,2.225081e+06,2225081,CA,ON,2020-11-05 21:12:29,NaT,NaN,1,0,0,2020-10-29 16:51:47,False,NaN,NaN,NaT
13491,7475289,2.153683e+06,2153683,US,CO,2020-11-16 04:11:43,NaT,NaN,1,1447,474,2020-09-30 23:11:41,True,business_account_3,NaN,NaT
13492,7144356,1.717608e+06,1717607,US,CT,2020-12-25 00:08:07,NaT,NaN,1,2187,824,2020-05-20 01:38:41,False,NaN,NaN,NaT
13493,7581194,2.243421e+06,2243420,US,IN,2020-11-19 16:43:04,NaT,NaN,4,2187,824,2020-11-05 20:02:09,False,NaN,NaN,NaT


In [24]:
# missing values of ADDRESS_TO_REGION 
orders[orders['ADDRESS_TO_REGION'].isna()]

,MERCHANT_ID,ORDER_ID,SHOP_ID,ADDRESS_TO_COUNTRY,ADDRESS_TO_REGION,ORDER_DT,FULFILLED_DT,REPRINT_FLAG,SALES_CHANNEL_TYPE_ID,TOTAL_COST,TOTAL_SHIPPING,MERCHANT_REGISTERED_DT,SUB_IS_ACTIVE_FLAG,SUB_PLAN,SHIPMENT_CARRIER,SHIPMENT_DELIVERD_DT
9,5900409,1.325831e+06,1325831,BE,NaN,2020-11-11 11:19:05,2020-11-12 02:44:37,NaN,1,2880,1180,2018-02-23 06:30:55,True,Plan 4,NaN,NaT
31,7520217,2.170880e+06,2170880,DK,NaN,2020-11-18 14:07:45,2020-11-24 14:39:25,NaN,1,889,400,2020-10-15 09:48:46,False,business_account_3,NaN,NaT
54,7271254,1.881174e+06,1881174,DE,NaN,2020-11-20 14:43:59,2020-11-23 16:23:32,NaN,4,1761,2690,2020-07-29 20:01:36,False,business_account_3,NaN,NaT
96,7501649,2.148728e+06,2148728,DE,NaN,2020-11-17 17:34:43,2020-11-20 02:36:14,NaN,2,1540,1200,2020-10-08 15:49:59,False,business_account_3,GLOBEGISTICS,NaT
153,6932458,1.449381e+06,1449381,GB,NaN,2020-12-05 19:10:06,2020-12-13 15:10:16,NaN,1,1024,800,2019-12-11 22:10:14,True,Plan 4,UPS_MAIL_INNOVATIONS,2021-01-22 15:13:23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13445,7575948,2.237140e+06,2237140,DE,NaN,2020-12-08 14:06:09,NaT,NaN,2,0,0,2020-11-03 21:27:27,False,NaN,NaN,NaT
13449,7087148,1.657683e+06,1657683,FI,NaN,2020-11-16 07:11:09,NaT,NaN,4,0,0,2020-04-09 22:27:02,False,NaN,NaN,NaT
13469,7169489,1.750016e+06,1750015,DE,NaN,2020-11-17 20:26:13,NaT,NaN,1,591,550,2020-06-06 07:38:57,False,NaN,NaN,NaT
13470,7169489,1.750015e+06,1750015,DE,NaN,2020-12-26 16:23:50,NaT,NaN,1,591,640,2020-06-06 07:38:57,False,NaN,NaN,NaT


In [25]:
# Order data timespan

order_dt_span = orders['ORDER_DT'].max() - orders['ORDER_DT'].min()
order_dt_span

Timedelta('90 days 23:27:05')

In [26]:
# Merchant Registering timespan

merchant_reg_span = orders['MERCHANT_REGISTERED_DT'].max() - orders['MERCHANT_REGISTERED_DT'].min()
merchant_reg_span

Timedelta('1406 days 20:00:48.657000')

In [27]:
# Merchant Registering timespan

merchant_reg_span = orders['MERCHANT_REGISTERED_DT'].max() - orders['MERCHANT_REGISTERED_DT'].min()
merchant_reg_span

Timedelta('1406 days 20:00:48.657000')

In [28]:
# Validate data entry with datetime entries
# The MERCHANT_REGISTERED_DT entry should be later than ORDER_DT for all entries
# If zero none of the merchant made an order before they registered on Printify
len(orders[orders['MERCHANT_REGISTERED_DT'] > orders['ORDER_DT']])

0

## TO describe the dates data, 
I'll use Sunburst charts from https://plotly.com/python/pie-charts/

In [29]:
# Unique sales channel types
# No number 5, interesting
orders['SALES_CHANNEL_TYPE_ID'].unique()

array([ 9,  4,  1,  2,  6,  3,  7, 11], dtype=int64)

In [30]:
# Look for null values in total cost
len(orders[orders['TOTAL_COST'].isna()])

0

In [31]:
# number for null values in total shipping cost
len(orders[orders['TOTAL_SHIPPING'].isna()])

0

In [32]:
# number for null values in SUB_IS_ACTIVE_FLAG
len(orders[orders['SUB_IS_ACTIVE_FLAG'].isna()])

0

In [33]:
# number for True values in SUB_IS_ACTIVE_FLAG
len(orders[orders['SUB_IS_ACTIVE_FLAG']])

9834

In [34]:
# number of orders that have reached the shpment carriers 
len(orders['SHIPMENT_CARRIER']) - len(orders[orders['SHIPMENT_CARRIER'].isna()])

11824

In [35]:
# number of orders that have been delivered
len(orders['SHIPMENT_DELIVERD_DT']) - len(orders[orders['SHIPMENT_DELIVERD_DT'].isna()])

10905

## Where is the shipment?
From above 2 cells, total shipments that have been dispatched to a carrier (11824) > total delivered orders (10905). The difference here are the orders that are on transit (919)

#### Assuming that date in SHIPMENT_DELIVERD_DT means the order is finalized

In [36]:
# Average number of orders per merchant

avg_merchant_orders = len(orders['ORDER_ID'].unique()) / len(orders['MERCHANT_ID'].unique())
avg_merchant_orders

2.8959227467811157

In [37]:
orders.describe()

,MERCHANT_ID,ORDER_ID,SHOP_ID,REPRINT_FLAG,SALES_CHANNEL_TYPE_ID,TOTAL_COST,TOTAL_SHIPPING
count,1.349500e+04,1.349500e+04,1.349500e+04,103.0,13495.000000,13495.000000,13495.000000
mean,6.878885e+06,1.559375e+06,1.559375e+06,1.0,2.759615,1386.652019,757.867358
std,7.227307e+05,5.447457e+05,5.447457e+05,0.0,1.529518,2165.980615,682.656422
min,2.650000e+02,4.210329e+04,4.210300e+04,1.0,1.000000,0.000000,0.000000
25%,6.713346e+06,1.358754e+06,1.358753e+06,1.0,2.000000,643.000000,424.000000
50%,7.068558e+06,1.705020e+06,1.705020e+06,1.0,2.000000,941.000000,574.000000
75%,7.239443e+06,1.915714e+06,1.915713e+06,1.0,4.000000,1483.000000,850.000000
max,7.729238e+06,2.423596e+06,2.423596e+06,1.0,11.000000,94228.000000,22200.000000


In [38]:
line_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57583 entries, 0 to 57582
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ORDER_ID           57583 non-null  float64
 1   PRINT_PROVIDER_ID  57583 non-null  int64  
 2   PRODUCT_BRAND      57583 non-null  object 
 3   PRODUCT_TYPE       57583 non-null  object 
 4   ITEM_STATUS        57583 non-null  object 
 5   QUANTITY           57583 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 2.6+ MB


In [39]:
line_items

,ORDER_ID,PRINT_PROVIDER_ID,PRODUCT_BRAND,PRODUCT_TYPE,ITEM_STATUS,QUANTITY
0,1.750015e+06,30,Generic brand,Mug,on-hold,3
1,1.750015e+06,30,Generic brand,Mug,on-hold,2
2,1.225155e+06,16,Gildan,Kids clothes,shipment_delivered,1
3,1.828891e+06,39,Bella+Canvas,T-Shirt,shipment_delivered,1
4,1.847848e+06,10,Generic brand,Accessories,shipment_delivered,1
...,...,...,...,...,...,...
57578,1.750015e+06,30,Generic brand,Mug,on-hold,1
57579,1.732949e+06,26,Gildan,T-Shirt,fulfilled,1
57580,1.449381e+06,29,Gildan,Sweatshirt,shipment_out_for_delivery,1
57581,1.735807e+06,48,Generic brand,Mug,shipment_delivered,1


In [40]:
len(line_items[line_items['ITEM_STATUS'] == 'canceled'])

1168

In [41]:
len(line_items['ORDER_ID'].unique())

13466

## Why is unique order ID here (13466) < unique order id is orders (13494)
Where is the 28 orders

In [42]:
line_items['QUANTITY'].unique()

array([  3,   2,   1,   8,  10,   6,   5,   4,  16,   7,  11,  12,  40,
        15,  30,  25, 100], dtype=int64)

In [43]:
len(line_items[line_items['ORDER_ID'] == 1629379.9])

10

In [44]:
len(line_items[line_items[['ORDER_ID', 'PRINT_PROVIDER_ID', 'PRODUCT_BRAND', 'PRODUCT_TYPE', 'ITEM_STATUS', 'QUANTITY']].duplicated()])

41775

In [45]:
len(line_items['ORDER_ID'].unique())

13466

In [46]:
len(line_items['PRINT_PROVIDER_ID'].unique())

39

In [47]:
line_items['PRINT_PROVIDER_ID'].unique()

array([30, 16, 39, 10, 45, 29, 25,  3, 33,  1, 59, 36, 41, 43, 26,  2, 14,
       51, 48, 44,  9,  6, 42, 65, 23, 28, 34,  7, 38, 27, 52, 37, 54, 47,
       40, 70,  8, 46, 49], dtype=int64)

In [48]:
# Remove the exact same rows in the df
duplicate_orders2 = line_items[line_items[['ORDER_ID', 'PRINT_PROVIDER_ID', 'PRODUCT_BRAND', 'PRODUCT_TYPE', 'ITEM_STATUS', 'QUANTITY']].duplicated()]

In [49]:
# Remove all duplicates in all columns and keep the first instance

# for index in duplicate_orders2.index:
#     line_items.drop(index, inplace=True)

line_items.drop_duplicates(subset=['ORDER_ID', 'PRINT_PROVIDER_ID', 'PRODUCT_BRAND', 'PRODUCT_TYPE', 'ITEM_STATUS', 'QUANTITY'], inplace=True)
line_items.reset_index(drop=True, inplace=True)
line_items

,ORDER_ID,PRINT_PROVIDER_ID,PRODUCT_BRAND,PRODUCT_TYPE,ITEM_STATUS,QUANTITY
0,1.750015e+06,30,Generic brand,Mug,on-hold,3
1,1.750015e+06,30,Generic brand,Mug,on-hold,2
2,1.225155e+06,16,Gildan,Kids clothes,shipment_delivered,1
3,1.828891e+06,39,Bella+Canvas,T-Shirt,shipment_delivered,1
4,1.847848e+06,10,Generic brand,Accessories,shipment_delivered,1
...,...,...,...,...,...,...
15803,1.735807e+06,2,Generic brand,Canvas,shipment_delivered,1
15804,1.732949e+06,26,Gildan,T-Shirt,fulfilled,1
15805,1.449381e+06,29,Gildan,Sweatshirt,shipment_out_for_delivery,1
15806,1.735807e+06,48,Generic brand,Mug,shipment_delivered,1


In [50]:
line_items[line_items['ORDER_ID'].duplicated()].sort_values(['ORDER_ID'])

,ORDER_ID,PRINT_PROVIDER_ID,PRODUCT_BRAND,PRODUCT_TYPE,ITEM_STATUS,QUANTITY
860,7.162613e+04,3,Gildan,Sweatshirt,shipment_delivered,1
8789,7.267617e+04,33,Generic brand,All Over Prints,shipment_delivered,1
3090,1.019216e+05,3,Gildan,Hoodie,shipment_delivered,1
3089,1.019216e+05,2,Generic brand,Poster,fulfilled,1
8643,1.057521e+05,1,Generic brand,Mug,shipment_delivered,1
...,...,...,...,...,...,...
9724,2.373266e+06,29,Bella+Canvas,T-Shirt,shipment_delivered,1
9725,2.373266e+06,29,Bella+Canvas,T-Shirt,shipment_delivered,4
12549,2.376201e+06,65,Gildan,Hoodie,shipment_delivered,1
12550,2.376201e+06,16,Generic brand,Hats,shipment_delivered,1


In [51]:
# Interesting. 
len(line_items) - len(line_items[line_items['ORDER_ID'].duplicated()].sort_values(['ORDER_ID']))

13466

In [52]:
len(line_items['ORDER_ID'].unique())

13466

In [53]:
# By how much is the orders is large compared to line_items orders_ids
len(orders['ORDER_ID'].unique()) - len(line_items['ORDER_ID'].unique())

29

## No order_id in the orders is similar to line_items

## NB: 
       Order_id has the first part as the shop_id, I will remove that and compare again

In [54]:
for order in orders['ORDER_ID']:
    order1 = str(order)
    order1 = order1.split(".")
    order1 = order1[-1]
    print(f"{order} --> {order1}")

2013244.178 --> 178
1358429.695 --> 695
939390.193 --> 193
1856107.4 --> 4
1843904.4 --> 4
1477206.11268 --> 11268
1720307.14704 --> 14704
604434.108 --> 108
1813012.309 --> 309
1325831.102 --> 102
2040024.157 --> 157
2081223.927 --> 927
1468748.6566 --> 6566
752154.3 --> 3
825384.15 --> 15
1733789.148 --> 148
1734809.25 --> 25
2190265.1 --> 1
119270.6848 --> 6848
733812.81 --> 81
2254663.2 --> 2
1294851.6032 --> 6032
1782844.3667 --> 3667
123752.12341 --> 12341
1755350.455 --> 455
1660181.53 --> 53
2000571.453 --> 453
1855923.57 --> 57
1468748.7466 --> 7466
1706670.8 --> 8
2094464.195 --> 195
2170880.11 --> 11
1924998.35 --> 35
459660.6149 --> 6149
1317433.1541 --> 1541
1218828.3629 --> 3629
1772051.1088 --> 1088
2164783.3 --> 3
2022704.72 --> 72
1988242.78 --> 78
1808692.15 --> 15
2155139.55 --> 55
1363735.1051 --> 1051
2013972.1798 --> 1798
1870843.96 --> 96
1928137.1495 --> 1495
1833131.92 --> 92
1908354.41 --> 41
2084603.516 --> 516
1610302.18291 --> 18291
2080417.3 --> 3
1840718.

In [55]:
orders[orders['ORDER_ID'].isna()]

,MERCHANT_ID,ORDER_ID,SHOP_ID,ADDRESS_TO_COUNTRY,ADDRESS_TO_REGION,ORDER_DT,FULFILLED_DT,REPRINT_FLAG,SALES_CHANNEL_TYPE_ID,TOTAL_COST,TOTAL_SHIPPING,MERCHANT_REGISTERED_DT,SUB_IS_ACTIVE_FLAG,SUB_PLAN,SHIPMENT_CARRIER,SHIPMENT_DELIVERD_DT


In [56]:

for index, order_whole in enumerate(orders['ORDER_ID']):
    order = str(order_whole)
    order = order.split(".")
    # print(f"{order_whole}-->{order_id}")
    order_id = int(order[-1])
    orders['ORDER_ID'].replace(order_whole, order_id, inplace=True)


In [57]:
zer = []
for index, order_whole in enumerate(line_items['ORDER_ID']):
    order = str(order_whole)
    order = order.split(".")
    
    order_id = int(order[1])
    if order_id == 0:
        print(f"{order_whole}-->{order_id}")
        zer.append(index)
    line_items['ORDER_ID'].replace(order_whole, order_id, inplace=True)

1219.0-->0
17.0-->0
0.0-->0
347.0-->0
101728.0-->0
0.0-->0
0.0-->0
22.0-->0
0.0-->0
2933.0-->0
1.0-->0
1.0-->0
0.0-->0
38.0-->0
19.0-->0
0.0-->0
1405.0-->0
0.0-->0
1.0-->0
0.0-->0
0.0-->0
0.0-->0
0.0-->0
0.0-->0
0.0-->0
161.0-->0
324.0-->0
1514.0-->0
852.0-->0
733.0-->0
401.0-->0
5634.0-->0
59.0-->0
10799.0-->0
0.0-->0
0.0-->0
2379.0-->0
0.0-->0
207.0-->0
11396.0-->0
1739.0-->0
11.0-->0
0.0-->0
4.0-->0
0.0-->0
1031.0-->0
114.0-->0
9.0-->0
439.0-->0
3186.0-->0
6476.0-->0
5.0-->0
37.0-->0
121.0-->0
36928.0-->0
144.0-->0
9498.0-->0
1141.0-->0
162.0-->0
103.0-->0
2519.0-->0
552.0-->0
95.0-->0
0.0-->0
1583.0-->0
0.0-->0
54207.0-->0
2045.0-->0
82.0-->0
646.0-->0
1.0-->0
32.0-->0
0.0-->0
5.0-->0
1303.0-->0
0.0-->0
0.0-->0
244.0-->0
273.0-->0
928.0-->0
1575.0-->0
132.0-->0
494.0-->0
1.0-->0
0.0-->0
16972.0-->0
0.0-->0
587.0-->0
17546.0-->0
69.0-->0
0.0-->0
23.0-->0
1.0-->0
0.0-->0
1314.0-->0
124.0-->0
879.0-->0
109389.0-->0
0.0-->0
0.0-->0
2.0-->0
0.0-->0
0.0-->0
102.0-->0
0.0-->0
12435.0-->0


In [58]:
print(zer)

[13, 24, 25, 45, 67, 68, 69, 74, 75, 90, 94, 98, 99, 103, 114, 115, 143, 144, 148, 149, 150, 151, 152, 153, 154, 165, 170, 183, 185, 198, 210, 245, 249, 256, 257, 258, 272, 273, 278, 287, 293, 327, 328, 376, 377, 389, 398, 411, 418, 426, 489, 500, 505, 507, 518, 526, 539, 541, 573, 576, 583, 593, 603, 604, 618, 619, 628, 635, 638, 642, 650, 653, 654, 682, 701, 702, 703, 726, 735, 738, 751, 753, 760, 772, 773, 786, 787, 795, 805, 822, 823, 830, 834, 835, 860, 871, 873, 892, 893, 894, 897, 898, 899, 905, 906, 918, 938, 941, 952, 969, 972, 973, 997, 998, 999, 1000, 1001, 1008, 1018, 1019, 1020, 1021, 1040, 1042, 1044, 1052, 1054, 1061, 1062, 1063, 1076, 1086, 1088, 1091, 1092, 1093, 1094, 1102, 1104, 1111, 1112, 1142, 1143, 1154, 1155, 1156, 1157, 1158, 1169, 1172, 1175, 1189, 1190, 1207, 1209, 1217, 1239, 1240, 1272, 1296, 1305, 1306, 1311, 1319, 1322, 1339, 1364, 1369, 1387, 1394, 1400, 1405, 1408, 1409, 1414, 1417, 1418, 1425, 1430, 1432, 1433, 1434, 1435, 1463, 1464, 1465, 1466, 1472,

In [59]:
line_items

,ORDER_ID,PRINT_PROVIDER_ID,PRODUCT_BRAND,PRODUCT_TYPE,ITEM_STATUS,QUANTITY
0,15208.0,30,Generic brand,Mug,on-hold,3
1,12357.0,30,Generic brand,Mug,on-hold,2
2,10081.0,16,Gildan,Kids clothes,shipment_delivered,1
3,707.0,39,Bella+Canvas,T-Shirt,shipment_delivered,1
4,0.0,10,Generic brand,Accessories,shipment_delivered,1
...,...,...,...,...,...,...
15803,1367.0,2,Generic brand,Canvas,shipment_delivered,1
15804,7852.0,26,Gildan,T-Shirt,fulfilled,1
15805,11854.0,29,Gildan,Sweatshirt,shipment_out_for_delivery,1
15806,0.0,48,Generic brand,Mug,shipment_delivered,1


In [60]:
orders

,MERCHANT_ID,ORDER_ID,SHOP_ID,ADDRESS_TO_COUNTRY,ADDRESS_TO_REGION,ORDER_DT,FULFILLED_DT,REPRINT_FLAG,SALES_CHANNEL_TYPE_ID,TOTAL_COST,TOTAL_SHIPPING,MERCHANT_REGISTERED_DT,SUB_IS_ACTIVE_FLAG,SUB_PLAN,SHIPMENT_CARRIER,SHIPMENT_DELIVERD_DT
0,7143578,178.0,2013244,US,WA,2020-10-02 19:30:15,2020-10-20 07:53:06,NaN,9,2902,2200,2020-05-19 15:22:57,True,business_account_3,UPS,2020-10-08 21:07:29
1,5888534,695.0,1358429,US,MN,2020-11-21 05:19:03,2020-11-23 00:23:32,NaN,4,1149,924,2018-02-08 07:34:02,True,business_account_3,USPS,2020-11-28 21:50:56
2,6484224,193.0,939390,US,AK,2020-10-25 01:39:08,2020-10-26 20:30:53,NaN,1,977,400,2019-04-09 20:02:46,True,business_account_3,USPS,2020-11-02 23:37:54
3,7251033,4.0,1856107,US,PA,2020-10-08 16:35:21,2020-10-10 15:02:47,NaN,1,661,800,2020-07-20 17:04:41,True,business_account_3,UPS,2020-10-15 18:08:37
4,7241488,4.0,1843904,US,CO,2020-11-17 19:30:10,2020-11-18 03:45:25,NaN,2,719,424,2020-07-15 18:47:51,True,business_account_3,USPS,2020-11-24 00:44:56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13490,7562898,12.0,2225081,CA,ON,2020-11-05 21:12:29,NaT,NaN,1,0,0,2020-10-29 16:51:47,False,NaN,NaN,NaT
13491,7475289,2224.0,2153683,US,CO,2020-11-16 04:11:43,NaT,NaN,1,1447,474,2020-09-30 23:11:41,True,business_account_3,NaN,NaT
13492,7144356,5.0,1717607,US,CT,2020-12-25 00:08:07,NaT,NaN,1,2187,824,2020-05-20 01:38:41,False,NaN,NaN,NaT
13493,7581194,66.0,2243420,US,IN,2020-11-19 16:43:04,NaT,NaN,4,2187,824,2020-11-05 20:02:09,False,NaN,NaN,NaT


In [61]:
# Check orders that are not in line_items dataset
dff = []
for order in line_items['ORDER_ID']:
    if order not in orders['ORDER_ID']:
        dff.append(order)
len(dff)

835

In [62]:
orders[orders['ORDER_ID'] == 0.0]

,MERCHANT_ID,ORDER_ID,SHOP_ID,ADDRESS_TO_COUNTRY,ADDRESS_TO_REGION,ORDER_DT,FULFILLED_DT,REPRINT_FLAG,SALES_CHANNEL_TYPE_ID,TOTAL_COST,TOTAL_SHIPPING,MERCHANT_REGISTERED_DT,SUB_IS_ACTIVE_FLAG,SUB_PLAN,SHIPMENT_CARRIER,SHIPMENT_DELIVERD_DT


In [63]:
len(line_items['ORDER_ID'].unique())

4766

In [64]:
line_items.sort_values(['ORDER_ID'])

,ORDER_ID,PRINT_PROVIDER_ID,PRODUCT_BRAND,PRODUCT_TYPE,ITEM_STATUS,QUANTITY
7903,0.0,43,Gildan,T-Shirt,shipment_in_transit,1
8127,0.0,48,Generic brand,Mug,shipment_delivered,1
8129,0.0,1,Generic brand,Mug,shipment_delivered,1
8131,0.0,29,Gildan,T-Shirt,fulfilled,1
8136,0.0,3,Bella+Canvas,T-Shirt,shipment_delivered,1
...,...,...,...,...,...,...
3858,115663.0,43,Next Level,T-Shirt,shipment_delivered,1
7382,116219.0,29,Bella+Canvas,T-Shirt,shipment_delivered,1
339,119746.0,1,Generic brand,All Over Prints,shipment_delivered,1
8411,120165.0,1,Generic brand,Mug,shipment_delivered,1


In [65]:
joined_df = pd.merge(orders, line_items, on=['ORDER_ID'])

In [66]:
print(len(orders['ORDER_ID'].unique()))

5315
